範例題目:<br>
1. 認識類別資料
2. 實做缺值處理方法與應用函式


範例重點:<br>
1. 類別資料，有分順序型與一般型，使用的編碼方式不同
2. 缺失值有很多處理方式，在這邊簡單介紹2種常見的方式

# 認識類別資料
變數的特徵屬於非數值型態。需利用一組的標記、類別、性質或名稱以區別每個基本單位的特徵、屬性。無法以數值表示的統計資料，如航班編號、性別、學歷、旅遊同伴、頭髮顏色、宗教等。
類別資料中可以分為兩類:順序性與一般性兩種。
* 順序性：類別之間存在順序性，例如：衣服尺寸[XL,L,M]、長度[短,中,長]
* 一般性：類別之間沒有順序關係，例如：顏色[黃,綠,藍]、性別[男,女]
-------------------------------------------------------------------------------------------------------------------------------
大部分的模型都是基於數學運算，字串無法套入數學模型進行運算，在此先對其進行 encoding 編碼(將類別資料轉成數字)才能進一步對其做分析。
* 對於__順序性__的類別資料，需要有順序性的 encoding 方法，可以使用 sklearn 中的 LabelEncoder()。
* 對於__一般性__的類別資料，則不需要有順序的編碼，可以使用 pandas 中的 get_dummies()

## 順序性 LabelEncoder()
順序性類別資料，編碼也需要有順序性，將類別資料依序編碼由 0 到 n-1，其中 n 為類別總數，因此類別之間會有順序關係 0<1<2<….，排序依照 python 內建順序，可以藉由 ord() 查看內建順序。

In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame([['green', 'M', 'male', 'short'],
          ['red', 'L', 'female', 'normal'],
          ['blue', 'XL', 'male', 'long']])
df.columns =['color', 'size', 'sex', 'lenght']
df

,color,size,sex,lenght
0,green,M,male,short
1,red,L,female,normal
2,blue,XL,male,long


In [3]:
#順序性類別資料，編碼也需要有順序性，將類別資料依序編碼由0到n-1，其中n為類別總數，因此類別之間會有順序關係0<1<2<….
from sklearn.preprocessing import LabelEncoder
df['size_label'] = LabelEncoder().fit_transform(df['size'].values)
df

,color,size,sex,lenght,size_label
0,green,M,male,short,1
1,red,L,female,normal,0
2,blue,XL,male,long,2


In [4]:
#排序依照python內建順序，可以藉由ord()查看內建順序
ord('L'),ord('M')

(76, 77)

## 一般性 get_dummies()
get_dummies() 把資料表中的每個類別 對應的欄位，經過**One-hot Encoding**(一位有效編碼)，增加資料表的欄位代表所屬的類別，如下欄位 color 中有 green、red、blue 將他們一一編入欄位中 color_blue、color_green、color_red 這個一對一的關係通常稱為 One-hot Encoding(一位有效編碼) 是沒有順序性的編碼。

In [5]:
#get_dummies()把資料表中的每個類別 對應的欄位，經過One-hot Encoding(一位有效編碼)
#One-hot Encoding(一位有效編碼) 是沒有順序性的編碼
pf = pd.get_dummies(df[['color']])
df = pd.concat([df, pf], axis=1)
df

,color,size,sex,lenght,size_label,color_blue,color_green,color_red
0,green,M,male,short,1,0,1,0
1,red,L,female,normal,0,0,0,1
2,blue,XL,male,long,2,1,0,0


# 認識缺值處理方法與應用函式
資料缺失時常發生在問卷資料上，填寫人時常會漏寫或不願意填寫，導致資料上有缺失值，只要缺失值將會填上 nan 代替缺失值，大部分的模型不能處理缺失值的問題，一般來說會將有缺失值的資料整筆直接刪除，但是這樣會損失其它欄位的資料，所以如果缺失情況不嚴重，傾向於將缺失值補上數值，以下最常見兩種補值方式。
* 定值補值：將缺失值都補上一個定值
* 前(後)補值：補前(後)一列的值

定值補值 補0、平均數、中位數

In [6]:
temp_data = pd.DataFrame([['2020-11-01', 24.8], 
              ['2020-11-02', 24.8],
              ['2020-11-03', None],
              ['2020-11-04', 25]],columns=['date','current_temp'])
temp_data

,date,current_temp
0,2020-11-01,24.8
1,2020-11-02,24.8
2,2020-11-03,NaN
3,2020-11-04,25.0


In [7]:
#以0填補
temp_data.fillna(0)

,date,current_temp
0,2020-11-01,24.8
1,2020-11-02,24.8
2,2020-11-03,0.0
3,2020-11-04,25.0


In [8]:
#以該欄位所有資料的算術平均數做填補
temp_data.fillna(temp_data.current_temp.mean())

,date,current_temp
0,2020-11-01,24.800000
1,2020-11-02,24.800000
2,2020-11-03,24.866667
3,2020-11-04,25.000000


In [9]:
#以該欄位所有資料的中位數做填補
temp_data.fillna(temp_data.current_temp.median())

,date,current_temp
0,2020-11-01,24.8
1,2020-11-02,24.8
2,2020-11-03,24.8
3,2020-11-04,25.0


前(後)補值最常使用在金融上，有時候因為颱風天導致沒有開盤，這時沒開盤那天的數值空了通常都會補前一天的價錢。

函式一樣使用 fillna()，我們只需要進一步運用參數 method=‘ffill’ 即可填補前一列數值， method=‘bfill’ 填補後一列數值。


In [10]:
#運用參數method=‘ffill’即可填補前一列數值
temp_data.fillna(method='ffill')

,date,current_temp
0,2020-11-01,24.8
1,2020-11-02,24.8
2,2020-11-03,24.8
3,2020-11-04,25.0


In [11]:
#method=‘bfill’填補後一列數值
temp_data.fillna(method='bfill')

,date,current_temp
0,2020-11-01,24.8
1,2020-11-02,24.8
2,2020-11-03,25.0
3,2020-11-04,25.0


In [12]:
#內差法補值
temp_data.interpolate()

,date,current_temp
0,2020-11-01,24.8
1,2020-11-02,24.8
2,2020-11-03,24.9
3,2020-11-04,25.0
